In [5]:
from tqdm.notebook import tqdm 
import numpy as np
import pandas as pd
import pickle


In [6]:
unprocessed_file_name = 'unprocessed_file.pkl'
cnn = "/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/summarisation/cnn/stories"

In [7]:

def read_file(filename):
    file = open(filename, encoding= 'UTF-8')
    text = file.read()
    file.close()
    return text

text = read_file('/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/summarisation/cnn/stories/0a0a4c90d59df9e36ffec4ba306b4f20f3ba4acb.story')
#text.split("\n")


In [8]:
def read_file(filename):
    file = open(filename, encoding= 'UTF-8')
    text = file.read()
    file.close()
    return text

def split_text(article):
    indx = article.index('@highlight')
    story = article[:indx]
    highlight = article[indx:].split('@highlight')

    highlight = ". ".join([h.strip() for h in highlight if len(h)>0])
    return story,highlight

In [9]:
from os import listdir

def read_all(folder):
    dataset = list()

    for file in tqdm(listdir(folder)):
        filename = folder + '/' + file
        article = read_file(filename)
        story,highlight = split_text(article)

        dataset.append({'story':story, 'highlight':highlight})
    
    return dataset

dataset = read_all(cnn)

  0%|          | 0/92579 [00:00<?, ?it/s]

In [10]:
#saving dataset for cleaning
import pickle
output_file = open(unprocessed_file_name,'wb')
pickle.dump(dataset, output_file)
output_file.close()

In [11]:
df = pd.DataFrame(pd.read_pickle(unprocessed_file_name))

In [12]:
df['word_count_text'] = df['story'].apply(lambda x: len(str(x).split()))
df['highlight_count'] = df['highlight'].apply(lambda x: len(str(x).split()))

In [16]:
from math import floor
print("The mean word count length of text article is " + str(floor(df['word_count_text'].mean())))
print("The mean word count length of summary/highlight is " + str(floor(df['highlight_count'].mean())))

The mean word count length of text article is 654
The mean word count length of summary/highlight is 41


In [17]:

print("The max word count length of text article is " + str(floor(df['word_count_text'].max())))
print("The max word count length of summary/highlight is " + str(floor(df['highlight_count'].max())))

The max word count length of text article is 1879
The max word count length of summary/highlight is 106
